# Modelos ART utilizando a função VAT

## Importando bibliotecas

In [ ]:
!pip install artlib
!pip install pyclustertend

In [5]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import USPS
from artlib import FuzzyART, FuzzyARTMAP, FusionART
from torch.utils.data import DataLoader
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, adjusted_rand_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from pyclustertend import vat

## Aplicando VAT

O VAT exige um X de 2 dimensões e isso conflita com o modelo ART.

Por isso, decidi separar essa parte do código.

In [6]:
transform = transforms.ToTensor()
full_dataset = datasets.USPS(root='./data', train=True, download=True, transform=transform)

X_list = []
y_list = []

for img, label in full_dataset:
    X_list.append(img.numpy().squeeze())  # shape (16,16)
    y_list.append(label)

X = np.array(X_list)  # (7291, 16, 16)
y = np.array(y_list)  # (7291,)
print("X:", X.shape, " y:", y.shape)

# Achatando e normalizando dados
# Isso é necessário para o scaler
nsamples, h, w = X.shape
X_flat = X.reshape((nsamples, h * w))  # (7291, 256)

scaler = StandardScaler()
X_flat_scaled = scaler.fit_transform(X_flat)

# Dividindo entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X_flat_scaled, y, test_size=0.4, random_state=42, stratify=y
)
print("Treino:", X_train.shape, "Teste:", X_test.shape)

X_sample = resample(X_train, n_samples=2000, random_state=42)

ODM = vat(X_sample, return_odm=True)
print("Matriz ODM (ordered dissimilarity):", ODM.shape)

NameError: name 'datasets' is not defined

## Baixando e ordenando dados

In [7]:
# Aprendizado não supervisionado
train_data = USPS(root='./USPS/', train=True, download=True)
test_data = USPS(root='./USPS/', train=False, download=True)

X_train = train_data.data
y_train = torch.tensor(train_data.targets, dtype=torch.long)

X_test = test_data.data
y_test = torch.tensor(test_data.targets, dtype=torch.long)

print(X_train.shape)

# as imagens do conjunto são 16x16
n_dim = 16 * 16

#flatten images
X_train = X_train.reshape(-1, n_dim)
X_test = X_test.reshape(-1, n_dim)

print(X_train.shape)

# Ordenando dados
sorted_indices_train = torch.argsort(y_train)
X_train_sorted = X_train[sorted_indices_train]
y_train_sorted = y_train[sorted_indices_train]

sorted_indices_test = torch.argsort(y_test)
X_test_sorted = X_test[sorted_indices_test]
y_test_sorted = y_test[sorted_indices_test]

100%|██████████| 6.58M/6.58M [00:00<00:00, 6.63MB/s]
100%|██████████| 1.83M/1.83M [00:00<00:00, 2.48MB/s]


(7291, 16, 16)
(7291, 256)


## Fuzzy ART

Aprendizado não supervionado


In [8]:
fuzzy_art_model = FuzzyART(rho=0.3, alpha=0.0, beta=1.0)

# Estabelecendo bounds
lower_bounds = np.zeros(n_dim)
upper_bounds = np.full(n_dim, 255.0)

fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)

# Normalização e complement coding
train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train)
test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test)

In [9]:
fuzzy_art_model.fit(train_X_fuzzy_art)
fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

In [10]:
ARI = adjusted_rand_score(y_test,fuzzy_art_predictions)
print(f'O ARI do modelo FuzzyART foi:{ARI}')
num_clusters = fuzzy_art_model.n_clusters
print(f"O número de clusters foi:{num_clusters}")

O ARI do modelo FuzzyART foi:0.04754222620721928
O número de clusters foi:284


## Matriz de acurácia

In [16]:
# Atenção!!!
# Para que funcione para outras bases de dados, será necessário ajustar a definição dos limites

def train_fuzzyART(X_train_subset, y_train_subset, X_test_subset, y_test_subset):
  fuzzy_art_model = FuzzyART(rho=0.3, alpha=0.0, beta=1.0)
  fuzzy_art_model.set_data_bounds(lower_bounds, upper_bounds)

  train_X_fuzzy_art = fuzzy_art_model.prepare_data(X_train_subset)
  test_X_fuzzy_art  = fuzzy_art_model.prepare_data(X_test_subset)

  fuzzy_art_model.fit(train_X_fuzzy_art)
  fuzzy_art_predictions = fuzzy_art_model.predict(test_X_fuzzy_art)

  return adjusted_rand_score(y_test_subset,fuzzy_art_predictions)

def generate_acc_matrix_fuzzyART(num_tasks):
  train_subsets = []
  test_subsets = []

  acc_matrix = [[0 for _ in range(num_tasks)] for _ in range(num_tasks)]

  for i in range(num_tasks):
    for j in range(num_tasks):
        # Classes até a i-ésima (inclusive)
        train_classes = torch.arange(0, i + 1)

        # Máscara de seleção para treino: todas as classes <= i
        mask_train = torch.isin(y_train_sorted, train_classes)
        X_train_subset = X_train_sorted[mask_train]
        y_train_subset = y_train_sorted[mask_train]

        # Máscara de seleção para teste: apenas a classe j
        mask_test = (y_test_sorted == j)
        X_test_subset = X_test_sorted[mask_test]
        y_test_subset = y_test_sorted[mask_test]

        # Armazena os subconjuntos (opcional)
        train_subsets.append((X_train_subset, y_train_subset))
        test_subsets.append((X_test_subset, y_test_subset))

        # print(f"Iteração {i}, {j}:")
        # print(f"  Treino -> classes até {i}, tamanho {len(X_train_subset)}\n")
        # print(f"  Treino -> classes até {i}, tamanho {y_train_subset.unique()}\n")
        # print(f"  Teste  -> classe {j}, tamanho {len(X_test_subset)}\n")
        # print(f"  Teste  -> classe {j}, tamanho {y_test_subset.unique()}\n")

        acc_matrix[i][j] = train_fuzzyART(
                            X_train_subset,
                            y_train_subset,  # não é usado na função
                            X_test_subset,
                            y_test_subset)
    return acc_matrix

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [18]:
classes = y_train.unique()
print(classes)
num_tasks = len(classes)

# for i in range (1, 6): 1 2 3 4 5

def average_accuracy_fuzzyART(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks+1)/2
  acc_sum = 0

  for i in range(num_tasks+1):
    for j in range(i+1):
      acc_sum = acc_sum + acc_matrix[i][j]

  return (acc_sum / denominator)

def backward_transfer_fuzzyART(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks-1)/2
  acc_sum = 0

  for i in range(2, num_tasks+1):
    for j in range(i):
      acc_sum = acc_sum + (acc_matrix[i][j] - acc_matrix[j][j])

  return (acc_sum / denominator)

def forward_transfer_fuzzyART(num_tasks, acc_matrix):
  denominator = num_tasks*(num_tasks-1)/2
  acc_sum = 0
  j = 0

  for i in range(j-1):
    for j in range(num_tasks+1):
      acc_sum = acc_sum + acc_matrix[i][j]

  return (acc_sum / denominator)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


## Fuzzy ARTMAP

In [ ]:
fuzzy_artmap_model = FuzzyARTMAP(rho=0.3, alpha=0.0, beta=1.0)

# Os limites foram definidos na parte do Fuzzy ART
fuzzy_artmap_model.module_a.set_data_bounds(lower_bounds, upper_bounds)

# Normalização e complement coding
train_X_fuzzy_artmap = fuzzy_artmap_model.prepare_data(X_train)
test_X_fuzzy_artmap = fuzzy_artmap_model.prepare_data(X_test)

In [ ]:
fuzzy_artmap_model.fit(train_X_fuzzy_artmap, y_train)
fuzzy_artmap_predictions = fuzzy_artmap_model.predict(test_X_fuzzy_artmap)

In [ ]:
report = classification_report(y_test, fuzzy_artmap_predictions)
matrix = confusion_matrix(y_test, fuzzy_artmap_predictions)

print("\nClassification report:")
print(report)

cm = confusion_matrix(y_test, fuzzy_artmap_predictions)
display = ConfusionMatrixDisplay(confusion_matrix=cm)
display.plot(cmap=plt.cm.Blues)
plt.title("Matriz de Confusão - FuzzyARTMAP")
plt.show()